# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [4]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [2]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze news and articles from" 
         "BBC, Washington Post, The Guardian," 
         "CNN, Wired, Scientific American, Harvard Business Review."
         "and summarize in a journalistic text to use in an English lesson.",
    backstory="Specializing in language teaching and journalism "
              "uses a progressive and anti-colonial perspective "
              "to provide crucial insights. With a knack for data,"
              "the Data Analyst Agent is the cornerstone for "
              "great lesson content.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
teaching_strategy_agent = Agent(
    role="Teaching Strategy Lesson Planner",
    goal="Develop and test various different activities based"
         "on the text from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of language teaching"
              "methods and student quantitative and qualitative analysis,"
              " this agent devises and refines activities for a defined text."
              "It evaluates the possible outcome of the lesson considering"
              "different teaching approaches to determine"
              "the most effective, engaging and interesting one.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
director_agent = Agent(
    role="Educative Content Creation Director",
    goal="Create a full lesson based on the content suggested by the"
         "Teaching Strategy Lesson Planner Agent and Data Analyst Agent"
         "based on approved language teaching methods.",
    backstory="This agent is a very successful educational book author "
              "that leverages his experience to organize the ideas from the"
              "team into complete lessons. The lessons always consider the"
              "student's level, goal and the progressiveness"
              "of activities in the lesson regarding efficiency and engagement.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [13]:
lesson_formatter_agent = Agent(
    role="Lesson Formater",
    goal="Format the lesson provided by the Educative Content Creation Director agent"
         "in a JSON format exactly as the director suggested",
    backstory="Armed with a deep understanding of data modeling "
              "json format, you will organize this lesson in objects. "
              "Each lesson object has a list of activity objects."
              "Each activity object has a list of element objects representing. "
              "the student tasks in the lesson.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Find a text and adapt it
data_analysis_task = Task(
    description=(
        "Look for a text about global news on topics aligned with"
        "the teacher input ({teacher_input})."
        "use your journalistic skills to find a text aligned with the topic"
        "and engaging for the student. After that implement your language"
        "teaching skills to adapt the text for the expected lesson goal"
    ),
    expected_output=(
        "A full adapted journalistic text extracted from the news channel citing the source"
        "how the adapted text can be applied for the scenario {teacher_input}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for teaching_strategy_agent: Create different activities
activity_creation_task = Task(
    description=(
        "Carefully analize the text generated by the Data Analyst Agent"
        "And suggest a list of multiple choice questions, open questions and "
        "conversation questions about the topic and the text. "
        "Consider the teacher input: ({teacher_input})."
    ),
    expected_output=(
        "A set of multiple choice quizzes, open questions and discussion questions "
        "that align with the topic, the text content and the grammar topics to develop."
    ),
    agent=teaching_strategy_agent,
)


In [11]:
# Task for Educative Content Creation Director: Plan the lesson
lesson_planning_task = Task(
    description=(
        "Analyze the text from the Data Analyst agent and the activities "
        "suggested by the teaching strategy lesson planner agent, "
        "considering current teacher input: ({teacher_input})."
    ),
    expected_output=(
        "A detailed file defining the lesson title, description, category,"
        "activity title, element title, element type, element task."
    ),
    agent=director_agent,
)


In [26]:
# Task for lesson_formatter_agent: Format the lesson
lesson_formatting_task = Task(
    description=(
        "Carefully format the lesson suggested by the Educative Content" 
        "Creation Director agent in a json format. The json file must be as follows: lesson=title, category, description, activities(array)=title, elements(array)=elementtitle,elementinstructions,elementtype,quiz_question(If Quiz),question(if Open Question), textbody(if conversation questions)'"
    ),
    expected_output=(
        "A json file representing the lesson correctly"
    ),
    agent=lesson_formatter_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [27]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
lesson_creation_crew = Crew(
    agents=[data_analyst_agent, 
            teaching_strategy_agent, 
            director_agent, 
            lesson_formatter_agent],
    
    tasks=[data_analysis_task, 
           activity_creation_task, 
           lesson_planning_task, 
           lesson_formatting_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.2),
    process=Process.hierarchical,
    verbose=True
)

2024-08-21 19:55:11,954 - 139893617310592 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [28]:
# Example data for kicking off the process
lesson_creation_inputs = {
    'teacher_input': 'Create a conversation lesson about the new AI regulation for my C1 student.',
    'CEFR_level': "C1"
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [29]:
### this execution will take some time to run
result = lesson_creation_crew.kickoff(inputs=lesson_creation_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Look for a text about global news on topics aligned withthe teacher input (Create a conversation lesson about the new AI regulation for my C1 student.).use your journalistic skills to find a text aligned with the topicand engaging for the student. After that implement your languageteaching skills to adapt the text for the expected lesson goal


> Entering new CrewAgentExecutor chain...
I need to delegate the task of finding a text about global news on topics aligned with the teacher input to the Educative Content Creation Director, as they have the skills to find engaging content for the student.

Action: Delegate work to co-worker
Action Input: {"coworker": "Educative Content Creation Director", "task": "Find a text about global news on topics aligned with the teacher input (Create a conversation lesson about the new AI regulation for my C1 student)", "context": "The text should be engaging and aligned with the topic of new

I need to gather information about the text regarding the new AI regulation in order to create appropriate questions for the lesson planner.

Action: Read website content
Action Input: {"website_url": "https://www.examplewebsite.com/new-ai-regulation"} 

Page not found – Example Website
Skip to content 
Example Website
Menu 
Menu 
Home
Blog
About
Contact
Oops! That page can’t be found.
It looks like nothing was found at this location. Maybe try searching?
Search for:
Search
RECENT POSTS
By 
Footer Widget 1
This is just an example text widget. You can add any kind of widget in here.
Mauris lacinia rhoncus lacus, eget bibendum ex gravida ac. Proin sed mauris ipsum. Sed eu commodo ipsum.
Footer Widget 2
This is just an example text widget. You can add any kind of widget in here.
Mauris lacinia rhoncus lacus, eget bibendum ex gravida ac. Proin sed mauris ipsum. Sed eu commodo ipsum.
Facebook
Twitter
Linkedin
© 2024 Example Website • Built with GeneratePress

Final Answer: Page not found – 

Action Input: {"search_query": "latest AI regulation updates 2021"} 


Search results: Title: AI Watch: Global regulatory tracker - United States | White & Case LLP
Link: https://www.whitecase.com/insight-our-thinking/ai-watch-global-regulatory-tracker-united-states
Snippet: Because global AI regulations remain in a constant state of flux, this AI Tracker will develop over time, adding updates and new jurisdictions ...
---
Title: Artificial intelligence (AI) act: Council gives final green light to the ...
Link: https://www.consilium.europa.eu/en/press/press-releases/2024/05/21/artificial-intelligence-ai-act-council-gives-final-green-light-to-the-first-worldwide-rules-on-ai/
Snippet: Today the Council approved a ground-breaking law aiming to harmonise rules on artificial intelligence, the so-called artificial intelligence ...
---
Title: EU Artificial Intelligence Act | Up-to-date developments and ...
Link: https://artificialintelligenceact.eu/
Snippet: The EU's AI regulation is already 

Final Answer: 
Based on my research, here are some engaging and interactive activities that align with the topic of AI regulation for an advanced student:

1. Enhance learning through gamification by using AI to create interactive activities like word walls, Jeopardy or game boards, scavenger hunts.
(Source: https://www.ascd.org/el/articles/using-ai-to-fuel-engagement-and-active-learning)

2. Try out fun activities like Beat ChatGPT and an AI Discovery Quest to get students using AI in a practical way.
(Source: https://www.weareteachers.com/ai-timely-topic/)

3. Engage in detailed conversations with AI to receive instant feedback, refine understanding, and practice explanation.
(Source: https://mitsloanedtech.mit.edu/ai/teach/getting-started/)

4. Use hands-on activities and games to make learning about AI interactive and engaging.
(Source: https://www.reddit.com/r/OpenAI/comments/1445zzp/how_can_i_teach_ai_in_my_classroom/)

5. Explore personalized AI activities that allow students to

I need to carefully format the lesson suggested by the Educative Content Creation Director in a json format to ensure it meets the criteria.

Action: Delegate work to co-worker
Action Input: {"coworker": "Lesson Formater", "task": "Format lesson in json format", "context": "Lesson on New AI Regulation for C1 Student: Welcome to today's conversation lesson on the topic of new AI regulation. Today, we will discuss the latest updates and regulations surrounding artificial intelligence in the European Union... (full context provided)"}

> Entering new CrewAgentExecutor chain...
I need to gather information about the lesson on new AI regulation for C1 students in order to format it in JSON format.

Action: Read website content
Action Input: {"website_url": "lesson_on_new_AI_regulation_for_C1_students.com"} 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'lesson_on_new_AI_regulation_for_C1_students.com': No scheme supplied. Perhaps you meant https://les

Thought: I now know the final answer
Final Answer: 
{
  "lesson": {
    "topic": "New AI Regulation for C1 Students",
    "description": "Welcome to today's conversation lesson on the topic of new AI regulation. Today, we will discuss the latest updates and regulations surrounding artificial intelligence in the European Union.",
    "activities": [
      {
        "title": "MEPs Approve World's First Comprehensive AI Law",
        "link": "https://www.linguahouse.com/esl-lesson-plans/legal-english/meps-approves-worlds-first-comprehensive-ai-law",
        "elements": [
          {
            "task": "Discuss the new comprehensive AI law approved by MEPs",
            "description": "Engage in a conversation about the implications of this new legislation"
          }
        ]
      },
      {
        "title": "AI and Ethics in Education",
        "link": "https://www.teachingenglish.org.uk/teaching-resources/teaching-secondary/lesson-plans/advanced-c1/ai-and-ethics-education",
        

- Display the final result as Markdown.

In [30]:
from IPython.display import Markdown
Markdown(result)

{
  "lesson": {
    "topic": "New AI Regulation for C1 Students",
    "description": "Welcome to today's conversation lesson on the topic of new AI regulation. Today, we will discuss the latest updates and regulations surrounding artificial intelligence in the European Union.",
    "activities": [
      {
        "title": "MEPs Approve World's First Comprehensive AI Law",
        "link": "https://www.linguahouse.com/esl-lesson-plans/legal-english/meps-approves-worlds-first-comprehensive-ai-law",
        "elements": [
          {
            "task": "Discuss the new comprehensive AI law approved by MEPs",
            "description": "Engage in a conversation about the implications of this new legislation"
          }
        ]
      },
      {
        "title": "AI and Ethics in Education",
        "link": "https://www.teachingenglish.org.uk/teaching-resources/teaching-secondary/lesson-plans/advanced-c1/ai-and-ethics-education",
        "elements": [
          {
            "task": "Analyze the ethical considerations of AI in education",
            "description": "Reflect on the impact of AI on educational practices and ethical dilemmas"
          }
        ]
      }
    ]
  }
}